# Galaxy Decomposition

In [1]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from multiprocessing import Pool

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [2]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings

In [3]:
figure_setup()

In [4]:
settings = Settings()

In [5]:
RXY_RANGE: tuple = (0, 200)
N_BINS: int = 200

In [6]:
def make_bins(rxy_min: float, rxy_max: float, n_bins: int):
    bin_edges = np.linspace(rxy_min, rxy_max, n_bins + 1)
    bin_width = np.diff(bin_edges)[0]
    bin_centers = bin_edges[1:] - bin_width / 2
    return bin_edges, bin_centers

In [7]:
def read_and_bin_data(simulation: str) -> tuple:
    """
    This method returns the circularity and the normalized potential for the
    stars in the main object of the simulation.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    tuple
        The circularity and normalized potential.
    """
    settings = Settings()

    s = Snapshot(simulation=simulation,
                 loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=settings.disc_std_circ,
        disc_min_circ=settings.disc_min_circ,
        cold_disc_delta_circ=settings.cold_disc_delta_circ,
        bulge_max_specific_energy=settings.bulge_max_specific_energy)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    region_tag = s.region_tag[is_real_star & is_main_obj]
    rxy = s.rho[is_real_star & is_main_obj]

    del s, is_real_star, is_main_obj

    # Create histogram of particle counts
    bin_edges = make_bins(RXY_RANGE[0], RXY_RANGE[1], N_BINS)[0]
    counts = np.array([
        np.histogram(a=rxy[region_tag == 0], bins=bin_edges,
                     range=RXY_RANGE)[0],
        np.histogram(a=rxy[region_tag == 1], bins=bin_edges,
                     range=RXY_RANGE)[0],
        np.histogram(a=rxy[region_tag == 2], bins=bin_edges,
                     range=RXY_RANGE)[0],
        np.histogram(a=rxy[region_tag == 3], bins=bin_edges,
                     range=RXY_RANGE)[0]])

    return counts

In [8]:
def make_sample_df(simulations: list) -> pd.DataFrame:
    settings = Settings()

    bin_centers = make_bins(RXY_RANGE[0], RXY_RANGE[1], N_BINS)[1]
    data = np.array(Pool().map(read_and_bin_data, simulations))
    data = data.reshape(data.shape[0] * data.shape[1], data.shape[2])

    columns = []
    for simulation in simulations:
        for component in settings.components:
            columns.append(f"{simulation}_StarCount_{component}")

    df = pd.DataFrame(data.T, columns=columns)
    df["BinCenters_ckpc"] = bin_centers
    return df

In [9]:
simulations = [f"au{i}_or_l4_s127" for i in settings.galaxies]
df = make_sample_df(simulations=simulations)

In [10]:
fig = plt.figure(figsize=(7.2, 7.2))
gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flat:
    ax.tick_params(which='both', direction="in")
    ax.set_yscale("log")
    ax.set_ylim(1E2, 1E6)
    ax.set_xlim(0, 100)
    ax.set_xticks([10, 30, 50, 70, 90])
    ax.set_yticks([1E3, 1E4, 1E5])
    ax.set_xlabel(r"$r_{xy}$ [ckpc]")
    ax.set_ylabel(r"$N$")
    ax.label_outer()

for i, simulation in enumerate(simulations):
    current_ax = axs.flatten()[i]
    current_ax.text(
        x=0.05, y=0.95, size=6.0,
        s=r"$\texttt{" + simulation.upper() + "}$",
        ha="left", va="top", transform=current_ax.transAxes)
    for component in settings.components:
        current_ax.plot(
            df["BinCenters_ckpc"],
            df[f"{simulation}_StarCount_{component}"],
            color=settings.component_colors[component],
            label=settings.component_labels[component])

axs[0, 0].legend(loc="center right", framealpha=0, fontsize=5.0)

fig.savefig(
    f"../images/galaxy_decomposition/distribution_of_particles.pdf")
plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(7.2, 7.2))
gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flat:
    ax.tick_params(which='both', direction="in")
    ax.set_yscale("log")
    ax.set_ylim(1E2, 1E6)
    ax.set_xlim(0, 100)
    ax.set_xticks([10, 30, 50, 70, 90])
    ax.set_yticks([1E3, 1E4, 1E5])
    ax.set_xlabel(r"$r_{xy}$ [ckpc]")
    ax.set_ylabel(r"$N$")
    ax.label_outer()

for i, simulation in enumerate(simulations):
    current_ax = axs.flatten()[i]
    current_ax.text(
        x=0.05, y=0.95, size=6.0,
        s=r"$\texttt{" + simulation.upper() + "}$",
        ha="left", va="top", transform=current_ax.transAxes)
    for component in settings.components:
        current_ax.plot(
            df["BinCenters_ckpc"],
            df[f"{simulation}_StarCount_{component}"],
            color=settings.component_colors[component],
            label=settings.component_labels[component])

axs[0, 0].legend(loc="center right", framealpha=0, fontsize=5.0)

fig.savefig(
    f"../images/galaxy_decomposition/distribution_of_particles.pdf")
plt.close(fig)